## Example of ML on Images:  Classifying Handwritten Digits

This extends the logistic regression example, and we'll abbreviate some of the code/cells at the beginning.

In [ ]:
import matplotlib.pyplot as plt
import sklearn.datasets
import sklearn.model_selection
import sklearn.metrics
import sklearn.preprocessing
import dtreeviz
import ipywidgets

In [ ]:
d = sklearn.datasets.load_digits()

x = d.data
y = d.target

print('We have %d samples, each with %d features' % (x.shape[0],
                                                     x.shape[1]))

x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(
        x, y, test_size=0.2, random_state=42, stratify=y)

scaler = sklearn.preprocessing.StandardScaler()
x_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

## Logistic Regression

In [ ]:
import sklearn.linear_model
lr_classifier = sklearn.linear_model.LogisticRegression()

In [ ]:
lr_classifier.fit(x_scaled, y_train)

In [ ]:
y_pred = lr_classifier.predict(x_test_scaled)

In [ ]:
print(f"Accuracy: {sklearn.metrics.accuracy_score(y_test, y_pred):.2%}")

In [ ]:
cm = sklearn.metrics.confusion_matrix(y_test, y_pred)
cm

# Decision tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
tree_clf = DecisionTreeClassifier()

In [ ]:
tree_clf.fit(x_train, y_train)

In [ ]:
tree_clf.classes_

In [ ]:
y_pred = tree_clf.predict(x_test)

In [ ]:
cm = sklearn.metrics.confusion_matrix(y_test, y_pred)
cm

In [ ]:
print(f"Accuracy: {sklearn.metrics.accuracy_score(y_test, y_pred):.2%}")

### Interpretation?

In [ ]:
text_representation = sklearn.tree.export_text(tree_clf)
print(text_representation)

In [ ]:
tree_clf.classes_

In [ ]:
plt.figure(figsize=(12,8))
sklearn.tree.plot_tree(tree_clf, 
               feature_names=range(64),  
               class_names=[str(i) for i in tree_clf.classes_],
               filled=True);

In [ ]:
%%capture --no-display

vizmodel = dtreeviz.model(tree_clf, 
         x,
         y,
         feature_names=range(64),
         class_names=[i for i in tree_clf.classes_],
         target_name="y")

vizmodel.view()

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier()

In [ ]:
rf_clf.fit(x_train, y_train)

In [ ]:
rf_clf.classes_

In [ ]:
y_pred = rf_clf.predict(x_test)

In [ ]:
cm = sklearn.metrics.confusion_matrix(y_test, y_pred)
cm

In [ ]:
print(f"Accuracy: {sklearn.metrics.accuracy_score(y_test, lr_classifier.predict(x_test_scaled)):.2%}")
print(f"Accuracy: {sklearn.metrics.accuracy_score(y_test, tree_clf.predict(x_test)):.2%}")
print(f"Accuracy: {sklearn.metrics.accuracy_score(y_test, rf_clf.predict(x_test)):.2%}")

Can we improve the Random Forest accuracy?

Actually, what parameters does it currently have?

In [ ]:
rf_clf.get_params()

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
cv_grid = GridSearchCV(RandomForestClassifier(n_jobs=-1,random_state=42),
                       param_grid = {
                           'max_depth' : [None,10,20],
                           'n_estimators' : [50,100,200],
                           'max_leaf_nodes' : [None,5,10]
                       })
cv_grid.fit(x_train, y_train)
cv_grid.best_params_

In [ ]:
y_pred = cv_grid.predict(x_test)

In [ ]:
cm = sklearn.metrics.confusion_matrix(y_test, y_pred)
cm

In [ ]:
print(f"Accuracy: {sklearn.metrics.accuracy_score(y_test, y_pred):.2%}")

### Interpretation?

In [ ]:
rf_clf.estimators_[0]

In [ ]:
def plttrees(t=0):
    plt.figure(figsize=(12,8))
    sklearn.tree.plot_tree(rf_clf.estimators_[t], 
               feature_names=range(64),  
               class_names=[str(i) for i in tree_clf.classes_],
               filled=True);

ipywidgets.interact(plttrees, t=range(len(rf_clf.estimators_)));

In [ ]:
rf_clf.feature_importances_

In [ ]:
plt.imshow(rf_clf.feature_importances_.reshape(8,8),
           cmap='binary')